In [ ]:
require("RPostgreSQL")
require("xts")
library(quantmod)
pw <- {
  ""
}
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "postgres",
                 host = "localhost", port = 5432,
                 user = "postgres")



Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.


In [195]:
instrument <- 'EUR_USD'
fromDate <- '2010-07-10 00:00:00'
toDate <- '2010-07-15 23:59:00'
period <- '5min'
fromtbl <- paste("oanda",instrument,"m1", sep = '_')

win_agg_group_sql <- paste("  with intervals as (
  select start, start + interval '", period ,"' as end
  from generate_series('" , fromDate , "', '", toDate,"', interval '", period ,"' ) as start
  )
  select distinct
  intervals.start as dt,
  min(bid_l) over w as low,
  max(bid_h) over w as high,
  first_value(bid_o) over w as open,
  last_value(bid_c) over w as close,
  sum(v) over w as volume
  from
  intervals
  join ",fromtbl," tmb on
  tmb.dt >= intervals.start and
  tmb.dt < intervals.end
  window w as (partition by intervals.start)
  order by intervals.start; "
)


fr <- dbGetQuery(con,win_agg_group_sql)


In [196]:
fr[,1] <- as.POSIXct(fr[,1],format="%Y")
# fr <- data.frame(fr[,-1], row.names=fr[,1])
#r <- xts(as.matrix(fr[,-1]), order.by=as.Date(fr[,1]), updated=Sys.time()) 
# fr <- xts(as.matrix(fr[,-1]), order.by=fr[,1])

# PRICE$DATE <- as.Date(as.character(PRICE$DATE),format="%Y%m%d")
fr[,1]

   [1] "2010-07-11 11:35:00 PDT" "2010-07-11 11:40:00 PDT"
   [3] "2010-07-11 11:45:00 PDT" "2010-07-11 11:50:00 PDT"
   [5] "2010-07-11 11:55:00 PDT" "2010-07-11 12:00:00 PDT"
   [7] "2010-07-11 12:05:00 PDT" "2010-07-11 12:10:00 PDT"
   [9] "2010-07-11 12:15:00 PDT" "2010-07-11 12:20:00 PDT"
  [11] "2010-07-11 12:25:00 PDT" "2010-07-11 12:30:00 PDT"
  [13] "2010-07-11 12:35:00 PDT" "2010-07-11 12:40:00 PDT"
  [15] "2010-07-11 12:50:00 PDT" "2010-07-11 12:55:00 PDT"
  [17] "2010-07-11 13:00:00 PDT" "2010-07-11 13:05:00 PDT"
  [19] "2010-07-11 13:10:00 PDT" "2010-07-11 13:15:00 PDT"
  [21] "2010-07-11 13:20:00 PDT" "2010-07-11 13:25:00 PDT"
  [23] "2010-07-11 13:30:00 PDT" "2010-07-11 13:35:00 PDT"
  [25] "2010-07-11 13:40:00 PDT" "2010-07-11 13:50:00 PDT"
  [27] "2010-07-11 13:55:00 PDT" "2010-07-11 14:00:00 PDT"
  [29] "2010-07-11 14:05:00 PDT" "2010-07-11 14:10:00 PDT"
  [31] "2010-07-11 14:15:00 PDT" "2010-07-11 14:20:00 PDT"
  [33] "2010-07-11 14:25:00 PDT" "2010-07-11 14:30:00 PD

In [176]:
fr

        high    open   close volume
<NA> 1.26327 1.26327 1.26297      2
<NA> 1.26359 1.26326 1.26351      4
<NA> 1.26354 1.26354 1.26351      4
<NA> 1.26354 1.26354 1.26351      4
<NA> 1.26370 1.26354 1.26365     15
<NA> 1.26379 1.26379 1.26210    107
<NA> 1.26213 1.26204 1.26188     21
<NA> 1.26306 1.26190 1.26290     84
<NA> 1.26283 1.26282 1.26247     29
<NA> 1.26257 1.26251 1.26230     13
<NA> 1.26226 1.26226 1.26145     89
<NA> 1.26158 1.26141 1.26136     20
<NA> 1.26142 1.26139 1.26135      8
<NA> 1.26156 1.26137 1.26156      9
<NA> 1.26198 1.26158 1.26182     45
<NA> 1.26204 1.26184 1.26200     21
<NA> 1.26207 1.26204 1.26202     24
<NA> 1.26197 1.26191 1.26180     19
<NA> 1.26193 1.26183 1.26177      8
<NA> 1.26197 1.26186 1.26189     24
<NA> 1.26288 1.26193 1.26221     48
<NA> 1.26240 1.26217 1.26214     42
<NA> 1.26250 1.26210 1.26250     21
<NA> 1.26267 1.26255 1.26266     21
<NA> 1.26266 1.26262 1.26266      5
<NA> 1.26266 1.26263 1.26266      2
<NA> 1.26335 1.26263 1.26335

[1] TRUE